<a href="https://colab.research.google.com/github/MrAdithya21/Real-Time-Reddit-Sentiment-Analysis/blob/main/json_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from azure.storage.blob import BlobServiceClient
import json
import pandas as pd

# 🔹 Azure Storage Account details
storage_account_name = "redditdatastream"  # Your storage account name
container_name = "reddit-data"  # The container where files are stored
folder_name = "processed"  # Folder where JSON files are stored
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-03-01T06:54:22Z&st=2025-02-23T22:54:22Z&spr=https&sig=J1RifDSIE4w4uMomjp8QRuH2vi0whNEvGv4qnVuOjHA%3D"  # Copy the generated SAS Token

# 🔹 Connect to Blob Storage
blob_service_client = BlobServiceClient(
    f"https://{storage_account_name}.blob.core.windows.net?{sas_token}")

container_client = blob_service_client.get_container_client(container_name)

# 🔹 List all JSON files (Ignore system-generated files)
blob_list = [blob for blob in container_client.list_blobs(name_starts_with=f"{folder_name}/")
             if not any(x in blob.name for x in ["_SUCCESS", "_committed", "_started"])]

data = []

# 🔹 Read and process JSON files
for blob in blob_list:
    print(f"Downloading: {blob.name}")
    blob_client = container_client.get_blob_client(blob.name)
    blob_data = blob_client.download_blob().readall()

    try:
        # 🔹 Read Multi-line JSON properly
        for line in blob_data.decode("utf-8").split("\n"):
            if line.strip():  # Ignore empty lines
                json_data = json.loads(line)
                data.append(json_data)
    except Exception as e:
        print(f"❌ Error parsing JSON file {blob.name}: {e}")

# 🔹 Convert to DataFrame
df = pd.DataFrame(data)

# 🔹 Display first 5 rows
print(df.head())

# 🔹 Save DataFrame to CSV (Optional)
df.to_csv("reddit_comments.csv", index=False)
print("✅ Data saved as 'reddit_comments.csv'")
